# A quickstart for Mitsuba building blocks

In [1]:
import drjit as dr
import mitsuba as mi

## 1. Random number generator: `mi.Sampler` class
A `mi.Sampler` instance can be easily understood as a random number generator.
`sampler.next_1d()` is similar to `np.random.rand()`, and `sampler.next_2d()` is similar to `np.random.rand(2)`, when `sampler` is an instance of `mi.Sampler` class.

Note that the resulting array shape for `sampler.next_1d()` and `sampler.next_2d()` are slightly depends on whether the Mitsuba variant is scalar or vectorized (CUDA or LLVM).

In [2]:
def test_sampler():
    sampler = mi.load_dict({'type': 'independent'})
    print(f"{type(sampler) = }")
    assert isinstance(sampler, mi.Sampler)
    print(sampler) # This shows some attributes for `sampler`,
                # but we have not to care about any of them now.

    val = sampler.next_1d()
    assert isinstance(val, mi.Float)
    print(f"sampler.next_1d() = {val} : {type(val)}")
    val = sampler.next_2d()
    assert isinstance(val, mi.Point2f)
    print(f"sampler.next_2d() = {val} : {type(val)}")

mi.set_variant('scalar_rgb')
print(f"#---------- {mi.variant()} ----------")
test_sampler()

mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')
print(f"\n#---------- {mi.variant()} ----------")
test_sampler()

#---------- scalar_rgb ----------
type(sampler) = <class 'mitsuba.scalar_rgb.Sampler'>
IndependentSampler[
  base_seed = 0
  sample_count = 4
  samples_per_wavefront = 1
  wavefront_size = 0
]
sampler.next_1d() = 0.10837864875793457 : <class 'float'>
sampler.next_2d() = [0.9069600105285645, 0.4066922664642334] : <class 'mitsuba.scalar_rgb.Point2f'>

#---------- cuda_ad_rgb ----------
type(sampler) = <class 'mitsuba.cuda_ad_rgb.Sampler'>
IndependentSampler[
  base_seed = 0
  sample_count = 4
  samples_per_wavefront = 1
  wavefront_size = 0
]
sampler.next_1d() = [0.10837864875793457] : <class 'drjit.cuda.ad.Float'>
sampler.next_2d() = [[0.9069600105285645, 0.4066922664642334]] : <class 'mitsuba.cuda_ad_rgb.Point2f'>


Note that common research-oriented renderers including Mitsuba uses the deterministic pseudo-random number generator PCG32. Thus, the same seed and the number of call of `next_1d()` and `next_2d()` yields the same results.

See more:
* https://www.pcg-random.org/index.html

We first test it for a scalar variant of Mitsuba 3

In [3]:
mi.set_variant('scalar_rgb')
sampler = mi.load_dict({'type': 'independent'})

sampler.seed(1)
print("\nThe seed has been set to be 1.")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")

sampler.seed(1)
print("\nThe seed has been set to be 1.")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")


The seed has been set to be 1.
sampler.next_1d() = 0.9390237331390381
sampler.next_1d() = 0.6919573545455933
sampler.next_1d() = 0.9697715044021606

The seed has been set to be 1.
sampler.next_1d() = 0.9390237331390381
sampler.next_1d() = 0.6919573545455933
sampler.next_1d() = 0.9697715044021606


For vectorized varitans in Mitsuba 3, we should specify a parameter `wavefront_size` to call `sampler.seed()`, which indicates the amount of vectorized data.

In [4]:
mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')
sampler = mi.load_dict({'type': 'independent'})

try:
    sampler.seed(1)
except RuntimeError as e:
    print("[ERROR OCCURED!]")
    print(e)

[ERROR OCCURED!]
​[Sampler] Sampler::seed(): wavefront_size should be specified!


In [5]:
sampler.seed(1, 1)
print("\nThe seed has been set to be 1.")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")

sampler.seed(1, 4)
print("\nThe seed has been set to be 1.")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_1d() = }")
print(f"{sampler.next_2d() = }")


The seed has been set to be 1.
sampler.next_1d() = [0.018652796745300293]
sampler.next_1d() = [0.5851906538009644]
sampler.next_1d() = [0.1382601261138916]

The seed has been set to be 1.
sampler.next_1d() = [0.018652796745300293, 0.9583619832992554, 0.7816940546035767, 0.8544584512710571]
sampler.next_1d() = [0.5851906538009644, 0.790186882019043, 0.670819878578186, 0.8393208980560303]
sampler.next_1d() = [0.1382601261138916, 0.8022022247314453, 0.23776817321777344, 0.9748337268829346]
sampler.next_2d() = [[0.294366717338562, 0.48410141468048096],
 [0.6647706031799316, 0.679118275642395],
 [0.6573762893676758, 0.7043564319610596],
 [0.011754870414733887, 0.20392751693725586]]
